In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [ ]:
# Define schema and plan parser as before
plan_schemas = [
    ResponseSchema(name="title", description="The job title the user wants to achieve."),
    ResponseSchema(name="time_frame", description="Time frame to reach the role (e.g., 2 years)."),
    ResponseSchema(name="skills", description="List of skills required.", type="list"),
    ResponseSchema(name="certifications", description="List of recommended certifications.", type="list"),
    ResponseSchema(name="milestones", description="Step-by-step roadmap milestones.", type="list")
]
plan_parser = StructuredOutputParser.from_response_schemas(plan_schemas)
plan_format_instructions = plan_parser.get_format_instructions()


In [ ]:
#structured_template: Function that builds a prompt for each domain
# (tech, marketing, finance) and embeds the formatting instructions and the user's input.

def structured_template(domain_description):
    return f"""
You are a career advisor for {domain_description}.
Analyze the user's goal and background. Generate a structured career plan with the following fields:

- title: Desired role
- time_frame: Expected duration
- skills: List of must-have skills
- certifications: List of relevant certifications
- milestones: Step-by-step roadmap

{{format_instructions}}

User Input:
{{input}}
"""


In [ ]:
prompt_infos = [
    {"name": "tech", "description": "technical roles like software engineer, cloud architect, data scientist"},
    {"name": "marketing", "description": "roles in product marketing, digital marketing, brand strategy"},
    {"name": "finance", "description": "finance roles like analyst, accountant, or investment banking"}
]

domain_prompts = {
    k['name']: ChatPromptTemplate.from_template(
        structured_template(k['description'])
    ).partial(format_instructions=plan_format_instructions)
    for k in prompt_infos
}

domain_chains = {
    name: prompt | llm | StrOutputParser()
    for name, prompt in domain_prompts.items()
}

In [ ]:
#Simple function to pick which advisor should handle the input, based on keywords.
def route_domain(inputs):
    user_input = inputs["input"].lower()
    if "marketing" in user_input:
        return "marketing"
    elif "finance" in user_input:
        return "finance"
    else:
        return "tech"


In [ ]:
#advisor_branch: Uses LCEL’s RunnableBranch to route the input to the right domain chain based on the routing function.
#(If no match, defaults to tech.)
advisor_branch = RunnableBranch(
    (lambda x: route_domain(x) == "tech", domain_chains["tech"]),
    (lambda x: route_domain(x) == "marketing", domain_chains["marketing"]),
    (lambda x: route_domain(x) == "finance", domain_chains["finance"]),
    domain_chains["tech"]
)

In [ ]:
plan_structured_chain = advisor_branch | RunnableLambda(lambda x: plan_parser.parse(x))
#

In [ ]:
encouragement_prompt = ChatPromptTemplate.from_template("""
You are a motivational career coach. Based on the following career goal and user plan, write an encouraging summary:

Goal: {input}
Target Role: {title}
Time Frame: {time_frame}
Skills Required: {skills}

Encouragement:
""")

In [ ]:
encouragement_chain = encouragement_prompt | llm | StrOutputParser()


In [ ]:
def lcel_pipeline(user_input):
    plan = plan_structured_chain.invoke({"input": user_input})
    encouragement = encouragement_chain.invoke({
        "input": user_input,
        "title": plan["title"],
        "time_frame": plan["time_frame"],
        "skills": ", ".join(plan["skills"])
    })
    return {
        "plan": plan,
        "encouragement": encouragement
    }


In [ ]:
# Usage
result = lcel_pipeline("I want to transition into product marketing within 2 years. I have a background in sales.")
print(result)

{'plan': {'title': 'Product Marketing Manager', 'time_frame': '2 years', 'skills': ['Market Research', 'Product Positioning', 'Go-to-Market Strategy', 'Cross-functional Collaboration', 'Data Analysis', 'Customer Segmentation', 'Content Creation', 'Digital Marketing'], 'certifications': ['Certified Product Marketing Manager (CPMM)', 'HubSpot Content Marketing Certification', 'Google Analytics Certification', 'Product Marketing Alliance Certification', 'Digital Marketing Institute Certification'], 'milestones': ['1. Conduct a self-assessment to identify transferable skills from sales to product marketing.', '2. Enroll in a product marketing course to gain foundational knowledge.', '3. Obtain at least one relevant certification (e.g., CPMM or HubSpot).', '4. Network with professionals in product marketing through LinkedIn and industry events.', '5. Gain experience by collaborating with product teams in your current role or through side projects.', '6. Start applying for product marketing 

In [ ]:
def beautify_output(result):
    plan = result['plan']
    encouragement = result['encouragement']

    print("\n **Career Plan:**\n")
    print(f"**Target Role:** {plan['title']}")
    print(f"**Time Frame:** {plan['time_frame']}\n")

    print("**Must-Have Skills:**")
    for skill in plan['skills']:
        print(f"- {skill}")
    print("\n**Recommended Certifications:**")
    for cert in plan['certifications']:
        print(f"- {cert}")
    print("\n**Step-by-Step Roadmap:**")
    for i, milestone in enumerate(plan['milestones'], 1):
        print(f"{i}. {milestone}")
    print("\n---\n")
    print("💡 **Encouragement:**\n")
    print(encouragement)

# Usage:
beautify_output(result)



 **Career Plan:**

**Target Role:** Product Marketing Manager
**Time Frame:** 2 years

**Must-Have Skills:**
- Market Research
- Product Positioning
- Go-to-Market Strategy
- Cross-functional Collaboration
- Data Analysis
- Customer Segmentation
- Content Creation
- Digital Marketing

**Recommended Certifications:**
- Certified Product Marketing Manager (CPMM)
- HubSpot Content Marketing Certification
- Google Analytics Certification
- Product Marketing Alliance Certification
- Digital Marketing Institute Certification

**Step-by-Step Roadmap:**
1. 1. Conduct a self-assessment to identify transferable skills from sales to product marketing.
2. 2. Enroll in a product marketing course to gain foundational knowledge.
3. 3. Obtain at least one relevant certification (e.g., CPMM or HubSpot).
4. 4. Network with professionals in product marketing through LinkedIn and industry events.
5. 5. Gain experience by collaborating with product teams in your current role or through side projects.
6. 6